<a href="https://colab.research.google.com/github/Julia-Schopper/DS-4002-Project-1/blob/main/The_New_Nation_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 79.9 MB/s eta 0:00:00


In [8]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Other tools
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from gensim.models import CoherenceModel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, defaultdict
/usr/local/lib/python3.8/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# read in data to google colab
from google.colab import files
uploades = files.upload()

Saving state_ofthe_union_texts.csv to state_ofthe_union_texts.csv


In [13]:
# Load in state of the union address data
import pandas as pd
# Read in data
url = "/content/state_ofthe_union_texts.csv"
df = pd.read_csv(url)
# The New Nation
df = df[(df["Year"]<1816)]
df

,President,Year,Title,Text
0,George Washington,1790,First State of the Union Address,['I embrace with great satisfaction the opport...
1,George Washington,1790,Second State of the Union Address,['Fellow-Citizens of the Senate and the House ...
2,George Washington,1791,Third State of the Union Address,['Fellow-Citizens of the Senate and the House ...
3,George Washington,1792,Fourth State of the Union Address,['Fellow-Citizens of the Senate and of the Hou...
4,George Washington,1793,Fifth State of the Union Address,['Fellow Citizens of the Senate and of the Hou...
5,George Washington,1794,Sixth State of the Union Address,['Fellow Citizens of the Senate and of the Hou...
6,George Washington,1795,Seventh State of the Union Address,['Fellow Citizens of the Senate and of the Hou...
7,George Washington,1796,Eighth State of the Union Address,['Fellow Citizens of the Senate and of the Hou...
8,by John Adams,1797,First State of the Union Address,['I was for some time apprehensive that it wou...
9,by John Adams,1798,Second State of the Union Address,['Gentlemen of the Senate and Gentlemen of the...


In [22]:
tokenizer = RegexpTokenizer(r'\w+[a-zA-Z]{3,}') #

# create English stop words list
stop_words = stopwords.words('english')
stop_words.extend(['shall', 'applause', 'nation', 'year', 'must', 'world', 'people', 'america', 'american', 'us', 'congress','u', 'y','gentlemen'])

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# compile sample documents into a list
# doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]
doc_set = df['Text'].values.tolist()

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in stop_words]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)


# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# filter extremes
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics=3, 
                                           id2word = dictionary, 
                                           passes=100)

for idx, topic in ldamodel.print_topics():
  print("Topic: {} \nWords: {}".format(idx, topic ))
  print("\n")
# print(ldamodel.print_topics(num_topics=10, num_words=10))

/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

Topic: 0 
Words: 0.017*"commission" + 0.009*"appoint" + 0.007*"ngentlemen" + 0.007*"majesti" + 0.006*"captur" + 0.006*"britain" + 0.006*"adjust" + 0.005*"institut" + 0.005*"noth" + 0.005*"tranquil"


Topic: 1 
Words: 0.017*"enemi" + 0.016*"command" + 0.008*"britain" + 0.008*"savag" + 0.007*"ship" + 0.007*"council" + 0.007*"instruct" + 0.007*"armi" + 0.006*"independ" + 0.006*"accept"


Topic: 2 
Words: 0.005*"friendli" + 0.005*"river" + 0.005*"take" + 0.005*"mississippi" + 0.005*"payment" + 0.005*"propos" + 0.004*"western" + 0.004*"quarter" + 0.004*"neighbor" + 0.004*"thought"




/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [23]:
# coherence of entire corpus, not each topic for each speech

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.286852334040579


In [24]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_table['Freq'] = token_table['Freq'].round()
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.030662 -0.099411       1        1  48.982951
1     -0.118039  0.027447       2        1  26.179184
0      0.087377  0.071963       3        1  24.837865, topic_info=           Term       Freq      Total Category  logprob  loglift
446  commission  31.000000  31.000000  Default  30.0000  30.0000
526       enemi  36.000000  36.000000  Default  29.0000  29.0000
445     command  43.000000  43.000000  Default  28.0000  28.0000
8       appoint  21.000000  21.000000  Default  27.0000  27.0000
585     britain  25.000000  25.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
585     britain  10.166596  25.613466   Topic3  -5.1792   0.4688
635      minist   8.853289  19.439665   Topic3  -5.3175   0.6063
324       river   8.007897  26.774539   Topic3  -5.4179   0.1858
53       fulfil   7.405752  15.508280   Topic3  -5.4960   0.6537
628      french   7.255350  16.956254   Topic3  -5.5166   0.5439

[185 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.988672    abroad
230       2  0.848912    accept
230       3  0.141485    accept
231       1  0.296867  accommod
231       2  0.148434  accommod
...     ...       ...       ...
228       1  0.368538    whilst
228       2  0.589661    whilst
501       1  0.922717      year
650       2  0.806991      york
650       3  0.161398      york

[293 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])